Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['Conversation-Entailment', '.ipynb_checkpoints', 'mistral_finetuned', 'logs', 'wandb', 'results', 'formatted_test.txt', 'formatted_test.jsonl', 'FinalProject.ipynb', 'ConvEnt_test_resplit.json', 'ConvEnt_train_resplit.json', 'precomputed_spans.json', 'results_roberta', 'ConvEnt_test_annotation_merged2.json', 'formatted_test_spans.json', 'www', 'probe_results', 'results_llama', 'formatted_test.json', 'results_llama_2', 'mistral']
/content/drive/MyDrive/Colab Notebooks/EECS595/FinalProj


In [3]:
!pip install transformers datasets torch evaluate bitsandbytes jsonlines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
!pip install torch==2.4.1

!pip install torchaudio==2.4.1 torchvision==0.19.1

!module load cuda/11.7.1

!pip install transformers datasets evaluate peft accelerate trl bitsandbytes einops flash_attn xformers==0.0.28.post1 wandb gradio sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from huggingface_hub import login
login("xxx")

In [ ]:
import json

input_file = "formatted_test.txt"
output_file = "formatted_test.json"

with open(input_file, "r") as infile, open(output_file, "w") as jsonl_outfile:
    buffer = []
    for line in infile:
        line = line.strip()
        if line:
            buffer.append(line)
        else:
            if buffer:
                  if len(buffer) >= 4 and buffer[0].startswith("Input:") and buffer[-1].startswith("Output:"):
                      dialogue = buffer[1].split("Dialogue: ")[1].strip('"')
                      hypothesis = buffer[2].split("Hypothesis: ")[1].strip('"')
                      label = int(buffer[-1].split("Output: ")[1].strip())

                      json_object = {
                          "dialogue": dialogue,
                          "hypothesis": hypothesis,
                          "label": label,
                      }

                      jsonl_outfile.write(json.dumps(json_object) + "\n")
                      buffer = []



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import Trainer
import torch.nn.functional as F
import wandb

wandb.login(key="xxx")
run = wandb.init(project='Finetuning Mistral-7B', job_type="training", anonymous="allow")
# Configuration
dataset_path = "ConvEnt_train_resplit.json"
#model_name = "mistralai/Mistral-7B-v0.3"
model_name = "meta-llama/Llama-2-7b-hf"
output_dir = "./llama2/3epoch"
#output_dir = "./results_llama_2"
num_train_epochs = 3
per_device_train_batch_size = 2
gradient_accumulation_steps = 8
learning_rate = 1e-5
logging_steps = 10
save_total_limit = 2
lora_r = 8
lora_alpha = 16
lora_dropout = 0.1
target_modules = ["q_proj", "v_proj"]

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bnb_config = None
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    num_labels=2,
    torch_dtype=torch.bfloat16,
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

lora_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="SEQ_CLS",
    init_lora_weights=True,
)
model = get_peft_model(model, lora_config)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        smooth_labels = torch.zeros_like(logits).scatter_(1, labels.unsqueeze(1), 1)
        smooth_labels = smooth_labels * 0.9 + 0.05

        loss = -torch.sum(F.log_softmax(logits, dim=1) * smooth_labels, dim=1).mean()
        return (loss, outputs) if return_outputs else loss

dataset = load_dataset("json", data_files={"train": dataset_path})["train"]

def format_dialogue(turns):
    dialogue = ""
    for turn in turns:
        speaker = turn["speaker"]
        text = turn["text"]
        dialogue += f"Speaker {speaker}: {text} "
    return dialogue.strip()

def preprocess_function(examples):
    dialogues = [format_dialogue(turns) for turns in examples["turns"]]
    tokenized = tokenizer(
        dialogues,
        examples["hypothesis"],
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors=None
    )
    tokenized["labels"] = examples["label"]
    return tokenized

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names
)

split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)

training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=logging_steps,
    save_total_limit=save_total_limit,
    fp16=False,
    bf16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    warmup_steps=100,
    gradient_checkpointing=True,
    max_grad_norm=0.5,
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
)

if model_name.startswith("mistralai"):
    for name, module in trainer.model.named_modules():
        if "norm" in name:
            module = module.to(torch.float32)

trainer.train()

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

@torch.no_grad()
def run_inference(dialogue, hypothesis):
    inputs = tokenizer(
        dialogue,
        hypothesis,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=256
    )
    outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1)
    return probs[0][1].item()


In [ ]:
import os
import json
import torch
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, SequentialSampler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from www.dataset.featurize import add_bert_features_ConvEnt, get_tensor_dataset
from www.dataset.prepro import get_ConvEnt_spans
from www.model.eval import evaluate, save_results, save_preds, load_preds
from www.utils import print_dict

# Configuration
seq_length = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "./llama2/3epoch"
#model_path = "./results_llama_2"
debug = False

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()

#merged_file = os.path.join("path_to_data", "ConvEnt_test_annotation_merged2.json")
ConvEnt_test = json.load(open("ConvEnt_test_annotation_merged2.json"))

ConvEnt_test = add_bert_features_ConvEnt(ConvEnt_test, tokenizer, seq_length, add_segment_ids=True)

ConvEnt_test_spans = get_ConvEnt_spans(ConvEnt_test)

ConvEnt_test_tensor = get_tensor_dataset(ConvEnt_test, label_key="label", add_segment_ids=True)
ConvEnt_test_spans_tensor = get_tensor_dataset(ConvEnt_test_spans, label_key="label", add_segment_ids=True)

test_sampler = SequentialSampler(ConvEnt_test_spans_tensor)
test_dataloader = DataLoader(ConvEnt_test_spans_tensor, sampler=test_sampler, batch_size=1)

metrics = [
    (accuracy_score, "accuracy")
]


def evaluate_no_token_type_ids(model, eval_dataloader, device, metrics):
    model.eval()
    all_labels = []
    all_preds = []

    for batch in eval_dataloader:
        input_ids, attention_mask, labels = [tensor.to(device) for tensor in batch]

        with torch.no_grad():
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
            )
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        labels = labels.cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels)

    results = {}
    for metric, name in metrics:
        results[name] = metric(all_labels, all_preds)

    return results, all_preds, all_labels


print("Evaluating model...")
results, preds, labels = evaluate_no_token_type_ids(model, test_dataloader, device, metrics)

probe_model = "./probe_results"
save_results(results, probe_model, "ConvEnt_test_spans")
labels_array = np.array(labels)
preds_array = np.array(preds)

save_preds([str(ex["example_id"]) for ex in ConvEnt_test_spans], labels_array, preds_array, probe_model, "ConvEnt_test_spans")

id_to_pred = {str(ex["example_id"]): pred for ex, pred in zip(ConvEnt_test_spans, preds)}
id_to_label = {str(ex["example_id"]): label for ex, label in zip(ConvEnt_test_spans, labels)}

lenient_coherence = []
strict_coherence = []

for example in ConvEnt_test:
    exid = str(example["example_id"])
    length = len(example["turns"])
    correct_spans = 0
    all_correct = True

    for sp1 in range(length):
        for sp2 in range(sp1, length):
            span_id = f"{exid}-sp{sp1}:{sp2}"
            pred = id_to_pred.get(span_id, 0)
            label = id_to_label.get(span_id, 0)

            if pred == label:
                correct_spans += 1
            else:
                all_correct = False

    total_spans = length * (length + 1) // 2
    lenient_coherence.append(correct_spans / total_spans if total_spans > 0 else 0)
    strict_coherence.append(1 if all_correct else 0)

results["lenient_coherence"] = sum(lenient_coherence) / len(lenient_coherence)
results["strict_coherence"] = sum(strict_coherence) / len(strict_coherence)

print("\nFinal Results:")
print_dict(results)
